In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from model import SiameseNet

from data_utils import QuoraDataset, DataIterator
from config import Config
from embeddings import load_embeddings

/home/bruno/Apps/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
import pandas as pd

In [20]:
labels, qids1, qids2, q1s, q2s = [], [], [], [], []

with open("data/MSRParaphraseCorpus/msr_paraphrase_train.txt", "r") as file:
    for i, line in enumerate(file):
        if i > 0:        
            y, qid1, qid2, q1, q2 = line.split("\t")
            labels.append(y)
            qids1.append(qid1)
            qids2.append(qid2)
            q1s.append(q1)
            q2s.append(q2)
            print(y, qid1, qid2, q1, q2)

1 702876 702977 Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence. Referring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.

0 2108705 2108831 Yucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion. Yucaipa bought Dominick's in 1995 for $693 million and sold it to Safeway for $1.8 billion in 1998.

1 1330381 1330521 They had published an advertisement on the Internet on June 10, offering the cargo for sale, he added. On June 10, the ship's owners had published an advertisement on the Internet, offering the explosives for sale.

0 3344667 3344648 Around 0335 GMT, Tab shares were up 19 cents, or 4.4%, at A$4.56, having earlier set a record high of A$4.57. Tab shares jumped 20 cents, or 4.6%, to set a record closing high at A$4.57.

1 1236820 1236712 The stock rose $2.11, or about 11 percent, to close Friday at $21.51 on the New York Stock Exchange. PG&E Corp. 

0 2100801 2100840 Just five months ago, Dean committed to accepting taxpayer money and vowed to attack any Democrat who didnt. Just five months ago, Dean committed to accepting taxpayer money and the spending limits that come with it and vowed to attack any Democrat who didnt.

1 2494709 2494434 "Contrary to the court's decision, we firmly believe Congress gave the FTC authority to implement the national Do Not Call list. "Contrary to the court's decision, we firmly believe Congress gave the F.T.C. authority to implement the national do-not-call list," the congressmen said in a statement.

1 2331916 2331805 Ruffner, 45, doesn't yet have an attorney in the murder charge, authorities said. Ruffner, 45, does not have a lawyer on the murder charge, authorities said.

1 1630585 1630657 Some of the computers also are used to send spam e-mail messages to drum up traffic to the sites. Some are also used to send spam e-mail messages to boost traffic to the sites.

1 1149320 1149378 It also face

0 1705190 1705093 It also named Robert Willumstad chief operating officer, also to take effect by Jan. 1, 2004. Robert B. Willumstad, 57, Citigroup's president, was named chief operating officer.

0 1831696 1831660 The agency charged that one WD Energy worker discussed false reporting with traders at two other energy companies. The agency found further that a WD Energy employee discussed false reporting with traders at two other energy companies, which the CFTC didn't identify.

0 259989 260054 "These despicable acts were committed by killers whose only faith is hate," Bush said, adding that the United States will find the killers. These despicable acts were committed by killers whose only faith is hate and the United States will find the killers and they will learn the meaning of American justice.

1 306378 306402 A second bomb, evidently carried by the other woman, did not explode; the authorities detonated the explosives later. A second bomb, evidently carried by the other woman, di

1 2654641 2654751 In 2001, President Bush named Kathleen Gregg one of six appointees to the Student Loan Marketing Association, the largest US lender for students. In 2001, President Bush named her to the Student Loan Marketing Association, the largest U.S. lender for students.

1 2328114 2328440 After hours of negotiations, a SWAT team burst in early yesterday and found Hoffine dead. Early Friday, the SWAT team burst in and found Hoffine dead.

1 2617727 2618289 "I never grabbed anyone and then pulled up their shirt and grabbed their breasts, and stuff like that. That I pulled up their shirt and grabbed their breasts and stuff like that: this is not me.

1 2720998 2720694 "The death of al-Ghozi signals that terrorists will never get far in the Philippines," Arroyo said in a statement on Monday. "The death of Ghozi signals that terrorists will never get far in the Philippines," President Gloria Arroyo said.

1 1460092 1459802 Board Chancellor Robert Bennett declined to comment on perso

1 2784756 2784682 Sanchez de Lozada promised a referendum on a controversial gas project, a reform of a free market energy law and constitutional reforms. Sanchez de Lozada promised a referendum on the gas project, a reform of energy laws and constitutional changes.

1 3347012 3347199 Yesterday, this honor was given to Firefighter Thomas C. Brick, who died last Tuesday battling a fire in an Upper Manhattan warehouse. They were there to honor Brick, who died Tuesday battling a four-alarm blaze in an upper Manhattan warehouse.

1 550074 550082 In the settlement, Tyrer agreed to pay $US1.06 million in allegedly ill-gotten trading profits and $US342,195 in interest, the SEC said. To settle the charges, Mr. Tyrer agreed to surrender $1.06 million in trading profits and $342,195 in interest.

1 1787563 1787470 The agency said it would not release the name of the 17-year-old involved because he is a minor. The agency said it had no plans to release the name of the teenager involved because he

In [21]:
print(len(labels), len(qids1), len(qids2), len(q1s), len(q2s))

4076 4076 4076 4076 4076


In [22]:
train_df = pd.DataFrame()
train_df["id"] = [i for i in range(len(labels))]
train_df["qid1"] = qids1
train_df["qid2"] = qids2
train_df["question1"] = q1s
train_df["question2"] = q2s
train_df["is_duplicate"] = labels

In [23]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1
1,1,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0
2,2,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1
3,3,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0
4,4,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1
5,5,738533,737951,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,1
6,6,264589,264502,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0
7,7,579975,579810,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,1
8,8,3114205,3114194,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0
9,9,1355540,1355592,He said the foodservice pie business doesn't f...,"""The foodservice pie business does not fit our...",1


In [26]:
labels, qids1, qids2, q1s, q2s = [], [], [], [], []

with open("data/MSRParaphraseCorpus/msr_paraphrase_test.txt", "r") as file:
    for i, line in enumerate(file):
        if i > 0:        
            y, qid1, qid2, q1, q2 = line.split("\t")
            labels.append(y)
            qids1.append(qid1)
            qids2.append(qid2)
            q1s.append(q1)
            q2s.append(q2)
            print(y, qid1, qid2, q1, q2)

1 1089874 1089925 PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will report directly to Mr So. Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So.

1 3019446 3019327 The world's two largest automakers said their U.S. sales declined more than predicted last month as a late summer sales frenzy caused more of an industry backlash than expected. Domestic sales at both GM and No. 2 Ford Motor Co. declined more than predicted as a late summer sales frenzy prompted a larger-than-expected industry backlash.

1 1945605 1945824 According to the federal Centers for Disease Control and Prevention (news - web sites), there were 19 reported cases of measles in the United States in 2002. The Centers for Disease Control and Prevention said there were 19 reported cases of measles in the United States in 2002.

0 1430402 1430329 A tropical storm rapidly developed in the Gulf of Mexico Sunday and was exp

1 1097469 1097432 Boeing said the final agreement is expected to be signed during the next few weeks. The Korean Air deal is expected to be finalized "in the next several weeks," Boeing spokesman Bob Saling said.

0 1383727 1383455 A cost analysis is under way, said Michael Rebell, CFE's executive director. "We're not looking for a Robin Hood remedy," said Michael Rebell, the campaign's executive director.

0 134822 134511 Bond bulls would like the Fed to recognize that risks are biased toward economic weakness. The Fed also said the risks to the economy were biased toward weakness.

0 1756866 1757157 In recent years, Hampton kept in touch with friends and stayed in trouble: He faced charges of fare-beating and credit card theft. In recent years, Mr. Hampton continued to run into trouble, facing charges of fare-beating and credit-card theft.

1 2410088 2410144 Five-year notes US5YT=RR rose 6/32 in price for a yield of 3.08 percent, down from 3.12 percent late on Tuesday. Five-year note

In [32]:
test_df = pd.DataFrame()
test_df["id"] = [i for i in range(len(train_df), len(train_df) + len(labels))]
test_df["qid1"] = qids1
test_df["qid2"] = qids2
test_df["question1"] = q1s
test_df["question2"] = q2s
test_df["is_duplicate"] = labels

In [33]:
test_df

,id,qid1,qid2,question1,question2,is_duplicate
0,4076,1089874,1089925,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...,1
1,4077,3019446,3019327,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...,1
2,4078,1945605,1945824,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,1
3,4079,1430402,1430329,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,0
4,4080,3354381,3354396,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...,0
5,4081,1390995,1391183,The settling companies would also assign their...,"Under the agreement, the settling companies wi...",1
6,4082,2201401,2201285,Air Commodore Quaife said the Hornets remained...,Air Commodore John Quaife said the security op...,0
7,4083,2453843,2453998,A Washington County man may have the countys f...,The countys first and only human case of West ...,1
8,4084,1756630,1756502,Moseley and a senior aide delivered their summ...,General Moseley and a senior aide presented th...,1
9,4085,938878,938896,The broader Standard & Poor's 500 Index <.SPX>...,The technology-laced Nasdaq Composite Index .I...,0


In [34]:
train_df.to_csv("data/MSRParaphraseCorpus/train.csv")

In [35]:
test_df.to_csv("data/MSRParaphraseCorpus/test.csv")